<a href="https://colab.research.google.com/github/JiaminJIAN/20MA573/blob/master/src/Importance_sampling_on_digital_put.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importance sampling**

### **Exercise:**

Asset price under $\mathbb{Q}$ follows 

$$S_{t} = S_{0} \exp \{\mu t + \sigma W_{t} \}.$$

Consider Digital put with its payoff

$$h(S_{T}) = I (S_{T} < S_{0} e^{-b}) .$$

We want to find the forward price:

$$v = \mathbb{E}^{\mathbb{Q}} [h(S_{T})].$$

Parameters are given as

$$r = 0.03, \sigma = 0.2, \mu = r - \frac{1}{2} \sigma^{2} = 0.01, T = 1, b = 0.39.$$

- Prove that the exact price is $0.02275$.
- Use OMC find the price.
- Use $IS(\alpha)$ find the price.
- Can you show your approach is optimal?
- Prove or demonstrate IS is more efficient to OMC.


## **Solution:**

(1) Prove that the exact price is $0.02275$.

Solution:

\begin{equation}
\begin{aligned}
v &= \mathbb{E}^{\mathbb{Q}} [h(S_{T})] \\
&= \mathbb{E}^{\mathbb{Q}} [I (S_{T} < S_{0} e^{-b})] \\
&= \mathbb{Q} (S_{T} < S_{0} e^{-b})\\
&= \mathbb{Q} (S_{0} \exp \{(r - \frac{1}{2} \sigma^{2}) T + \sigma W_{T}\} < S_{0} e^{-b}) \\
&= \mathbb{Q} (Z < - \frac{b + (r - \frac{1}{2} \sigma^{2}) T}{\sigma \sqrt{T}}) \\
&= \mathbb{Q} (Z < - 2) \\
&= \Phi(-2), \\
\end{aligned}
\end{equation}
where $Z \sim \mathcal N (0, 1)$ under probability measure $\mathbb{Q}$ and $\Phi(\cdot)$ is the culmulative distribution function of standard normal variable. Then we know the exact price of this forward is $\Phi(-2)$.

In [1]:
import numpy as np
import scipy.stats as ss

v = ss.norm.cdf(-2) 
print("The exact value of this forward is:", v)

The exact value of this forward is: 0.022750131948179195


(2) Use OMC find the price.

Solution:

Since we get $v= \Phi(-2) $ in (1), then we have the estimator

$$\hat{v}  \approx \frac{1}{n} \sum_{i = 1}^{n} I(X_{i} < -2),$$

where $X_{i} \sim \mathcal{N}(0, 1)$.

In [0]:
## Use monte carlo method to get the forward price

def monte_carlo_v(N):
  sum_v = 0
  for i in range(N):
    x = np.random.normal()
    if x < -2:
      sum_v += 1
  return sum_v/N

In [3]:
## Test the monte carlo method

v_hat = monte_carlo_v(10000)
print("The value of this forward by the monte carlo method is:", v_hat)

The value of this forward by the monte carlo method is: 0.0233


(3) Use $IS(\alpha)$ find the price.

Solution:

For the importance sampling , we suppose $\phi_{\alpha}(\cdot)$ is the probability distribution function of the Gaussian random variable with mean $-\alpha$ and variance $1$. Then we have

\begin{equation}
\begin{aligned}
v &= \Phi(-2) \\
&= \int_{- \infty}^{-2} \phi_{0}(x) \, d x \\
&= \int_{- \infty}^{-2} \frac{\phi_{0}(x)}{\phi_{\alpha}(x)} \phi_{\alpha}(x) \, d x \\
&= \int_{- \infty}^{-2} e^{\frac{1}{2} \alpha^{2} + \alpha x} \phi_{\alpha} (x) \, d x \\
&= \mathbb{E} [I(Y < - 2) e^{\frac{1}{2} \alpha^{2} + \alpha Y} | Y \sim \phi_{\alpha}] \\
&\approx \frac{1}{n} \sum_{i = 1}^{n} [I(Y_{i} < - 2) e^{\frac{1}{2} \alpha^{2} + \alpha Y_{i}}]. \\
\end{aligned}
\end{equation}

In [0]:
## Use importance sampling method to get the forward price

def importance_sampling_v(N, alpha):
  sum_v = 0
  for i in range(N):
    y = np.random.normal(- alpha, 1)
    if y < -2:
      sum_v += np.exp(0.5 * alpha ** 2 + alpha * y)
  return sum_v / N

In [5]:
## Test the importance sampling method

v_hat = importance_sampling_v(10000, 4)
print("The value of this forward by the importance sampling method is:", v_hat)

The value of this forward by the importance sampling method is: 0.022070269486359897


We can use the importance sampling method to generate a sequence of estimators, and then we can calculate the sample variance of this sequence. And the $\alpha$ which is correspondent to the minimum variance is the desired one. But this method may not very exactly.

In [6]:
estimate_list = [] 
variance_list = []
for i in range(10):
  for j in range(100):
    estimate_list.append(importance_sampling_v(1000, i))
  variance_list.append(np.var(estimate_list))

alpha_optimal1 = variance_list.index(min(variance_list))
print("The optimal choice of alpha is", alpha_optimal1)

The optimal choice of alpha is 3


In [7]:
v_hat = importance_sampling_v(10000, alpha_optimal1)
print("The value of this forward by the importance sampling method is:", v_hat)

The value of this forward by the importance sampling method is: 0.023079668609016963


(4) Can you show your approach is optimal?

Solution:

By the result we get from (3), we know that when $\alpha = 3$, the variance of the extimators by the importance sampling method is the minimum one. We can also calculate the varaince of the estimators. As 

$$\hat{v} = \frac{1}{n} \sum_{i = 1}^{n} [I(Y_{i} < - 2) e^{\frac{1}{2} \alpha^{2} + \alpha Y_{i}}]$$

by the importance sampling method. We know that 

\begin{equation}
\begin{aligned}
MSE(\hat{v}) &= var(\hat{v}) \\
&= \frac{1}{n} var(I(Y_{i} < - 2) e^{\frac{1}{2} \alpha^{2} + \alpha Y_{i}}) \\
&= \frac{1}{n} \{ \mathbb{E} [I(Y_{i} < - 2) e^{ \alpha^{2} + 2 \alpha Y_{i}}] - \Phi^{2}(-2) \}. \\
\end{aligned}
\end{equation}

As we have

\begin{equation}
\begin{aligned}
\mathbb{E} [I(Y_{i} < - 2) e^{ \alpha^{2} + 2 \alpha Y_{i}}] &= \int_{- \infty}^{-2} e^{\alpha^{2}+ 2 \alpha y}  \frac{1}{\sqrt{2 \pi}} e^{- \frac{(y + \alpha)^{2}}{2}} \, d y \\
&= \int_{- \infty}^{-2} \frac{1}{\sqrt{2 \pi}} e^{- \frac{y^{2} - \alpha y - \alpha^{2}}{2}} \, d y \\
&= \int_{- \infty}^{-2} \frac{1}{\sqrt{2 \pi}} e^{- \frac{(y - \alpha)^{2}}{2}}  e^{\alpha^{2}} \, d y  \\
&= e^{\alpha^{2}} \Phi(-2-\alpha),
\end{aligned}
\end{equation}

then we can get

$$MSE(\hat{v}) = \frac{1}{n} \{e^{\alpha^{2}} \Phi(-2-\alpha) - \Phi^{2}(-2)\}.$$

Thus we know that the desired $\alpha$ satisfies the equation:

$$2 \alpha \Phi(- 2 - \alpha) = \phi(-2 - \alpha).$$

In [8]:
## get the optimal alpha

mse_list = []
for i in range(10):
  mse_list.append(np.exp(i**2) * ss.norm.cdf(-2 - i) - ss.norm.cdf(-2) ** 2)

alpha_optimal2 = mse_list.index(min(mse_list))
print("The optimal choice of alpha is", alpha_optimal2)

The optimal choice of alpha is 2


In [9]:
v_hat = importance_sampling_v(10000, alpha_optimal2)
print("The value of this forward by the importance sampling method is:", v_hat)

The value of this forward by the importance sampling method is: 0.02335980963571127


(5) Prove or demonstrate IS is more efficient to OMC.


Solution:

By the Monte-Carlo method, we have

$$\hat{v_{1}} \approx \frac{1}{n} \sum_{i = 1}^{n} I(X_{i} < -2),$$

then the variance of this estimator is

\begin{equation}
\begin{aligned}
MSE(\hat{v_{1}}) &= var(\hat{v_{1}}) \\
&= \frac{1}{n} var(I(X_{i} < - 2)) \\
&= \frac{1}{n} \{ \mathbb{E} [I(X_{i} < - 2)] - (\mathbb{E} [I(X_{i} < - 2)])^{2} \} \\
&= \frac{1}{n} \{\Phi(-2) - (\Phi(-2))^{2}\}.
\end{aligned}
\end{equation}

For importance sampling method, we have proved that

$$MSE(\hat{v_{2}}) = \frac{1}{n} \{e^{\alpha^{2}} \Phi(-2-\alpha) - \Phi^{2}(-2)\},$$

hence we have

$$MSE(\hat{v_{1}}) - MSE(\hat{v_{2}}) = \frac{\Phi(-2) - e^{\alpha^{2}} \Phi(-2-\alpha)}{n}.$$

When $\alpha = 2$, the difference of MSE between these two method is

$$\frac{\Phi(-2) - e^{4} \Phi(-4)}{n}$$



In [10]:
## the difference of MSE between these two method 

distance = ss.norm.cdf(-2) - np.exp(4) * ss.norm.cdf(-4)
print("The difference of MSE between these two method is:", distance)

The difference of MSE between these two method is: 0.021020940734838522


Since $\frac{\Phi(-2) - e^{4} \Phi(-4)}{n} > 0$, we know that importance sampling method is more efficient to ordinary Monte-Carlo method.